In [ ]:
# full_script_supergauss_fwhm.py
import re
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit, least_squares
from scipy.signal import savgol_filter

# ---------------------------------------------------------------------
# --- USER: set base folder for your txt files
# ---------------------------------------------------------------------
FILE_BASE = r'C:/Users/sann7609/Documents/Oxford/ChannelAnalysis_CALA_Sept25_minisforum/HOFI_profiles/CALA_Sept25_channels_txt_files/'

# ---------------------------------------------------------------------
# --- I/O / small helpers
# ---------------------------------------------------------------------
def extract_columns(filename):
    """Load a whitespace text file (skip header row) and return columns as dict."""
    filepath = FILE_BASE + filename
    data = np.loadtxt(filepath, skiprows=1)
    return {
        'r_um': data[:, 0],
        'ne_r': data[:, 1],
        'n0_r_rel': data[:, 2]
    }

# ---------------------------------------------------------------------
# --- Gaussian utilities (used to seed supergaussian fits)
# ---------------------------------------------------------------------
def gauss(x, H, A, x0, sigma):
    return H + A * np.exp(-(x - x0) ** 2 / (2 * sigma ** 2))

def moment_guesses(x, y):
    """Robust moment-based initial guesses for Gaussian (H, A, x0, sigma)."""
    H = np.median(y)
    y0 = y - H
    flip = -1 if np.min(y0) < 0 else 1
    y1 = flip * y0
    y1[y1 < 0] = 0
    area = np.trapezoid(y1, x)
    dx = max(x[1] - x[0], 1e-9)
    if not np.isfinite(area) or area <= 0:
        x0 = x[np.argmin(y)] if flip == -1 else x[np.argmax(y)]
        sigma = max((x[-1] - x[0]) / 10.0, dx)
        A = (np.min(y) - H) if flip == -1 else (np.max(y) - H)
        return H, A, x0, abs(sigma)
    x0 = np.trapezoid(x * y1, x) / area
    var = np.trapezoid(((x - x0) ** 2) * y1, x) / area
    sigma = np.sqrt(max(var, dx ** 2))
    A = flip * np.max(y1)
    return H, A, x0, sigma

def fit_gaussian_robust(x, y):
    """
    Fit a Gaussian robustly: try curve_fit first, then least_squares fallback.
    Returns (params, cov) where cov is None if unavailable.
    """
    # light smoothing for stability of initial guesses
    if len(y) >= 7:
        window = 11 if len(y) >= 11 else (len(y)//2*2 + 1)
        y_sm = savgol_filter(y, window, 2, mode='interp')
    else:
        y_sm = y
    H0, A0, x00, s0 = moment_guesses(x, y_sm)
    dx = max(x[1] - x[0], 1e-9)
    lower = [np.min(y) - abs(np.ptp(y)), -np.inf, x.min(), dx]
    upper = [np.max(y) + abs(np.ptp(y)), np.inf, x.max(), (x[-1] - x[0])]
    p0 = [H0, A0, x00, max(s0, dx)]
    try:
        params, cov = curve_fit(gauss, x, y, p0=p0, bounds=(lower, upper), maxfev=10000)
        return params, cov
    except Exception:
        def resid(p):
            return gauss(x, *p) - y
        res = least_squares(resid, p0, bounds=(lower, upper), loss='soft_l1', f_scale=np.std(y) or 1.0, max_nfev=4000)
        return res.x, None

# ---------------------------------------------------------------------
# --- Supergaussian model and fit
# ---------------------------------------------------------------------
def supergauss(x, H, A, x0, w, n):
    """H + A * exp( - (|x-x0| / w) ** n )"""
    return H + A * np.exp(- (np.abs(x - x0) / w) ** n)

def fit_supergaussian_robust(x, y, p0=None):
    """Least-squares fit with soft L1 loss; returns params and the result object."""
    x = np.asarray(x)
    y = np.asarray(y)
    if p0 is None:
        H0 = np.min(y)
        A0 = np.max(y) - H0
        x0 = x[np.argmax(y)]
        w0 = (np.max(x) - np.min(x)) / 6.0
        n0 = 2.0
    else:
        H0, A0, x0, w0, n0 = p0
    lb = [-np.inf, 0.0, np.min(x), 1e-8, 0.5]
    ub = [ np.inf, np.inf, np.max(x), np.inf, 50.0]
    p_init = np.array([H0, A0, x0, w0, n0])
    res = least_squares(lambda p: supergauss(x, *p) - y,
                        p_init, bounds=(lb, ub),
                        loss='soft_l1', f_scale=0.1, max_nfev=15000)
    return res.x, res

# ---------------------------------------------------------------------
# --- FWHM calculator for our supergaussian parameterization
# ---------------------------------------------------------------------
def compute_FWHM_from_params(w, n):
    """FWHM = 2 * w * (ln 2)^(1/n) for H + A*exp(-(abs(x-x0)/w)**n)"""
    return 2.0 * w * (np.log(2.0) ** (1.0 / n))

# ---------------------------------------------------------------------
# --- Example datasets (edit/add your own lists here)
# ---------------------------------------------------------------------
filenames_100mbar_560 = [
    'tscan_250917_Bdel560_100mbar_t0p0ns.txt',
    'tscan_250917_Bdel560_100mbar_t1p0ns.txt',
    'tscan_250917_Bdel560_100mbar_t2p0ns.txt',
    'tscan_250917_Bdel560_100mbar_t2p5ns.txt',
    'tscan_250917_Bdel560_100mbar_t3p0ns.txt',
    'tscan_250917_Bdel560_100mbar_t3p5ns.txt',
    'tscan_250917_Bdel560_100mbar_t4p0ns.txt',
]
filenames_100mbar_582 = [
    'tscan_250917_Bdel582_100mbar_t0p0ns.txt',
    'tscan_250917_Bdel582_100mbar_t1p0ns.txt',
    'tscan_250917_Bdel582_100mbar_t2p0ns.txt',
    'tscan_250917_Bdel582_100mbar_t2p5ns.txt',
    'tscan_250917_Bdel582_100mbar_t3p0ns.txt',
    'tscan_250917_Bdel582_100mbar_t3p5ns.txt',
    'tscan_250917_Bdel582_100mbar_t4p0ns.txt',
]
filenames_80mbar_560 = [
    'tscan_250917_Bdel560_80mbar_t0p0ns.txt',
    'tscan_250917_Bdel560_80mbar_t1p0ns.txt',
    'tscan_250917_Bdel560_80mbar_t2p0ns.txt',
    'tscan_250917_Bdel560_80mbar_t2p5ns.txt',
    'tscan_250917_Bdel560_80mbar_t3p0ns.txt',
    'tscan_250917_Bdel560_80mbar_t3p5ns.txt',
    'tscan_250917_Bdel560_80mbar_t4p0ns.txt',
]
filenames_80mbar_582 = [
    'tscan_250917_Bdel582_80mbar_t0p0ns.txt',
    'tscan_250917_Bdel582_80mbar_t1p0ns.txt',
    'tscan_250917_Bdel582_80mbar_t2p0ns.txt',
    'tscan_250917_Bdel582_80mbar_t2p5ns.txt',
    'tscan_250917_Bdel582_80mbar_t3p0ns.txt',
    'tscan_250917_Bdel582_80mbar_t3p5ns.txt',
    'tscan_250917_Bdel582_80mbar_t4p0ns.txt',
]
filenames_60mbar_560 = [
    'tscan_250917_Bdel560_60mbar_t0p0ns.txt',
    'tscan_250917_Bdel560_60mbar_t1p0ns.txt',
    'tscan_250917_Bdel560_60mbar_t2p0ns.txt',
    'tscan_250917_Bdel560_60mbar_t2p5ns.txt',
    'tscan_250917_Bdel560_60mbar_t3p0ns.txt',
    'tscan_250917_Bdel560_60mbar_t3p5ns.txt',
    'tscan_250917_Bdel560_60mbar_t4p0ns.txt',
]
filenames_60mbar_582 = [
    'tscan_250917_Bdel582_60mbar_t0p0ns.txt',
    'tscan_250917_Bdel582_60mbar_t1p0ns.txt',
    'tscan_250917_Bdel582_60mbar_t2p0ns.txt',
    'tscan_250917_Bdel582_60mbar_t2p5ns.txt',
    'tscan_250917_Bdel582_60mbar_t3p0ns.txt',
    'tscan_250917_Bdel582_60mbar_t3p5ns.txt',
    'tscan_250917_Bdel582_60mbar_t4p0ns.txt',
]
filenames_40mbar_560 = [
    'tscan_250917_Bdel560_40mbar_t0p0ns.txt',
    'tscan_250917_Bdel560_40mbar_t1p0ns.txt',
    'tscan_250917_Bdel560_40mbar_t2p0ns.txt',
    'tscan_250917_Bdel560_40mbar_t2p5ns.txt',
    'tscan_250917_Bdel560_40mbar_t3p0ns.txt',
    'tscan_250917_Bdel560_40mbar_t3p5ns.txt',
    'tscan_250917_Bdel560_40mbar_t4p0ns.txt',
]
filenames_40mbar_582 = [
    'tscan_250917_Bdel582_40mbar_t0p0ns.txt',
    'tscan_250917_Bdel582_40mbar_t1p0ns.txt',
    'tscan_250917_Bdel582_40mbar_t2p0ns.txt',
    'tscan_250917_Bdel582_40mbar_t2p5ns.txt',
    'tscan_250917_Bdel582_40mbar_t3p0ns.txt',
    'tscan_250917_Bdel582_40mbar_t3p5ns.txt',
    'tscan_250917_Bdel582_40mbar_t4p0ns.txt',
]


# Registry: label -> (list_of_filenames, optional_titles_list)
datasets = {
    '100mbar_Bdel560': (filenames_100mbar_560, ['t = 0 ns','t = 1 ns','t = 2 ns', 't = 2.5 ns','t = 3 ns','t = 3.5 ns','t = 4 ns']),
    '100mbar_Bdel582': (filenames_100mbar_582, ['t = 0 ns','t = 1 ns','t = 2 ns', 't = 2.5 ns','t = 3 ns','t = 3.5 ns','t = 4 ns']),
    '80mbar_Bdel560': (filenames_80mbar_560, ['t = 0 ns','t = 1 ns','t = 2 ns', 't = 2.5 ns','t = 3 ns','t = 3.5 ns','t = 4 ns']),
    '80mbar_Bdel582': (filenames_80mbar_582, ['t = 0 ns','t = 1 ns','t = 2 ns', 't = 2.5 ns','t = 3 ns','t = 3.5 ns','t = 4 ns']),
    '60mbar_Bdel560': (filenames_60mbar_560, ['t = 0 ns','t = 1 ns','t = 2 ns', 't = 2.5 ns','t = 3 ns','t = 3.5 ns','t = 4 ns']),
    '60mbar_Bdel582': (filenames_60mbar_582, ['t = 0 ns','t = 1 ns','t = 2 ns', 't = 2.5 ns','t = 3 ns','t = 3.5 ns','t = 4 ns']),
    '40mbar_Bdel560': (filenames_40mbar_560, ['t = 0 ns','t = 1 ns','t = 2 ns', 't = 2.5 ns','t = 3 ns','t = 3.5 ns','t = 4 ns']),
    '40mbar_Bdel582': (filenames_40mbar_582, ['t = 0 ns','t = 1 ns','t = 2 ns', 't = 2.5 ns','t = 3 ns','t = 3.5 ns','t = 4 ns']),

}

# ---------------------------------------------------------------------
# --- User options: which datasets to plot and whether to show profiles
# ---------------------------------------------------------------------
selected = None       # None -> plot all datasets in `datasets`; or provide list of labels to plot
plot_profiles = False  # True -> overlay raw profiles with supergaussian fits
show_fwhm_plot = True # True -> plot FWHM vs time
xlim_for_fwhm = (0,4.5)   # e.g. (0,4.5) or None to autoscale
ylim_for_fwhm = (0,140)   # e.g. (0,130) or None to autoscale

# ---------------------------------------------------------------------
# --- Processing helper
# ---------------------------------------------------------------------
def process_dataset(filenames, titles_list=None, compute_profiles=False):
    times = []
    FWHMs = []
    profiles = []  # list of tuples (x, y, xs, fit_xs, fname, n)
    for i, fname in enumerate(filenames):
        cols = extract_columns(fname)
        x = cols['r_um']
        y = cols['ne_r'] / np.max(cols['ne_r'])

        # seed from gaussian fit
        params_gauss, _ = fit_gaussian_robust(x, y)
        H_g, A_g, x0_g, sigma_g = params_gauss
        w_guess = sigma_g * np.sqrt(2.0)
        p0_super = [H_g, A_g, x0_g, w_guess, 2.0]

        # supergauss fit
        params_sg, _ = fit_supergaussian_robust(x, y, p0=p0_super)
        H, A, x0, w, n = params_sg
        FWHM = compute_FWHM_from_params(w, n)

        # time extraction: prefer provided titles_list, else parse filename
        if titles_list is not None and i < len(titles_list):
            tstr = titles_list[i]
        else:
            tstr = fname
        m = re.search(r'(\d+(\.\d+)?)\s*ns', tstr)
        if m:
            times.append(float(m.group(1)))
        else:
            m2 = re.search(r'_t(\d+(\.\d+)?)ns', fname)
            times.append(float(m2.group(1)) if m2 else i)

        FWHMs.append(FWHM)

        if compute_profiles:
            xs = np.linspace(np.min(x), np.max(x), 400)
            fit_xs = supergauss(xs, *params_sg)
            profiles.append((x, y, xs, fit_xs, fname, n))

    return np.array(times), np.array(FWHMs), profiles

# ---------------------------------------------------------------------
# --- Run selected datasets, collect results
# ---------------------------------------------------------------------
all_labels = list(datasets.keys())
if selected is None:
    selected_labels = all_labels
else:
    selected_labels = [L for L in selected if L in datasets]

results = {}
for label in selected_labels:
    fnames, titles_list = datasets[label]
    times_arr, fwhms_arr, profiles = process_dataset(fnames, titles_list=titles_list, compute_profiles=plot_profiles)
    # sort by time
    order = np.argsort(times_arr)
    results[label] = (times_arr[order], fwhms_arr[order], [profiles[i] for i in order] if plot_profiles else [])

# ---------------------------------------------------------------------
# --- Final plotting (all at the end)
# ---------------------------------------------------------------------
if plot_profiles:
    plt.figure(figsize=(8,6))
    first_label_done = set()
    for label, (times_arr, fwhms_arr, profiles) in results.items():
        for x, y, xs, fit_xs, fname, n in profiles:
            plt.plot(x, y, '.', alpha=0.35)
            # label only once per dataset to avoid huge legends
            lbl = f'{label} fit (n={n:.2f})' if label not in first_label_done else None
            plt.plot(xs, fit_xs, '-' if lbl is None else '--', alpha=0.8, label=lbl)
            if lbl is not None:
                first_label_done.add(label)
    plt.xlabel('r (µm)')
    plt.ylabel('normalized n_e')
    plt.title('Profiles and supergaussian fits')
    plt.legend(loc='best', fontsize='small')
    plt.grid(True, alpha=0.2)
    plt.tight_layout()

if show_fwhm_plot:
    plt.figure(figsize=(7,5))
    for label, (times_arr, fwhms_arr, _) in results.items():
        plt.plot(times_arr, fwhms_arr, 'o-', label=label)
    plt.xlabel('time (ns)')
    plt.ylabel('FWHM (µm)')
    plt.title('FWHM vs time')
    plt.legend()
    plt.grid(True, alpha=0.25)
    if xlim_for_fwhm is not None:
        plt.xlim(*xlim_for_fwhm)
    if ylim_for_fwhm is not None:
        plt.ylim(*ylim_for_fwhm)
    else:
        # small padding in y
        try:
            ymin = min(np.min(v[1]) for v in results.values())
            ymax = max(np.max(v[1]) for v in results.values())
            dy = max(0.1*(ymax-ymin), 1e-9)
            plt.ylim(max(0, ymin - dy), ymax + dy)
        except Exception:
            pass
    plt.tight_layout()

# print numeric results
print("Dataset | times (ns) | FWHMs (µm)")
for label, (times_arr, fwhms_arr, _) in results.items():
    times_s = ", ".join([f"{t:.1f}" for t in times_arr])
    fwhm_s = ", ".join([f"{f:.2f}" for f in fwhms_arr])
    print(f"{label} | {times_s} | {fwhm_s}")

plt.show()